# Tutorial pyOSRD #6
Creating custom agent

In [1]:
import pandas as pd

from pyosrd.schedules import Schedule
from pyosrd.agents.scheduler_agent import SchedulerAgent


## Creating a dumb agent
Create an agent that add 60 seconds of delay at the second zone the first train go throught.

In [2]:

class DumbSchedulerAgent(SchedulerAgent):
    @property
    def regulated_schedule(self) -> Schedule:
        """As an example, make train1 stop 60s more in first zone"""
        return self.delayed_schedule.add_delay(0, 0, 60)


## Test our agent on all scenarii
We can launch our custom agent on all scenarii implemented in the module OSRD.with_delays.

In [3]:
from pyosrd.agents.scheduler_agent import regulate_scenarii_with_agents
from functools import partial
from pyosrd.schedules.weights import all_steps

agent = DumbSchedulerAgent("dumb")


regulate_scenarii_with_agents("all", agent)

[16:17:57,954] [INFO]  [StandaloneSimulationCommand] Loading infra: tmp/infra.json
[16:17:57,985] [INFO]              [FullInfra] parsing infra
[16:17:57,985] [INFO]              [FullInfra] adaptation to kotlin
[16:17:58,020] [INFO]              [FullInfra] loading signals
[16:17:58,023] [INFO]              [FullInfra] building blocks
[16:17:58,043] [INFO]  [StandaloneSimulationCommand] Loading input: tmp/simulation.json
[16:17:58,215] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.0
[16:17:58,315] [DEBUG]             [Allowance] computing range n°1
[16:17:58,317] [DEBUG]             [Allowance]   computing section n°1
[16:17:58,318] [DEBUG]             [Allowance]   target time = 310.84232970000005
[16:17:58,318] [DEBUG]             [Allowance]     starting attempt 1
[16:17:58,320] [DEBUG]             [Allowance]     envelope time 518.965036
[16:17:58,320] [DEBUG]             [Allowance]     starting attempt 2
[16:17:58,320] [DEBUG]             [Al

,dumb
c1_2trains_delay_train1,360.0
c1_with_blocks_2trains_delay,1410.0
c1_with_blocks_2trains_diffspeed_delay,1200.0
c1_with_blocks_3trains_delay,1410.0
c1y2_2trains_conflict,120.0
c1y2_2trains_no_conflict,220.0
c1y2y1_2trainsdiffstation_delay150,750.0
c1y2y1_2trainsdiffstation_delay50,450.0
c1y2y1_2trainssamestation_delay150,750.0
c1y2y1_2trainssamestation_delay50,450.0


## Compare two agents

The function regulate_scenarii_with_agents can be launched on multiple agents at the same time. It returns a pandas DataFrame containing the ponderated delays
for all scenarii.

An agent can be compared to another one using those scenarii.

In [4]:

class DumberSchedulerAgent(SchedulerAgent):
    @property
    def regulated_schedule(self) -> Schedule:
        """As an example, make train1 stop 120s more in second zone"""
        return self.delayed_schedule.add_delay(1, 0, 120)

To speed up the notebook execution the next call is done only on one scenario "c1_delay".

In [5]:

agent_dumber = DumberSchedulerAgent("dumber")
regulate_scenarii_with_agents("c1_2trains_delay_train1", [agent, agent_dumber])

[16:18:10,971] [INFO]  [StandaloneSimulationCommand] Loading infra: tmp/infra.json
[16:18:11,002] [INFO]              [FullInfra] parsing infra
[16:18:11,002] [INFO]              [FullInfra] adaptation to kotlin
[16:18:11,037] [INFO]              [FullInfra] loading signals
[16:18:11,040] [INFO]              [FullInfra] building blocks
[16:18:11,061] [INFO]  [StandaloneSimulationCommand] Loading input: tmp/simulation.json
[16:18:11,225] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.40
[16:18:11,336] [DEBUG]             [Allowance] computing range n°1
[16:18:11,338] [DEBUG]             [Allowance]   computing section n°1
[16:18:11,339] [DEBUG]             [Allowance]   target time = 310.84232970000005
[16:18:11,339] [DEBUG]             [Allowance]     starting attempt 1
[16:18:11,340] [DEBUG]             [Allowance]     envelope time 518.965036
[16:18:11,341] [DEBUG]             [Allowance]     starting attempt 2
[16:18:11,341] [DEBUG]             [A

,dumb,dumber
c1_2trains_delay_train1,360.0,480.0
